In [1]:
import boto3
import pandas as pd
from io import StringIO
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import json

In [3]:
BUCKET_NAME = "team1-index-predictor-bucket"

raw_data_filename = "data/raw/sp500.csv"
raw_inference_data_filename = "data/raw/sp500-inference.csv"

s3 = boto3.client("s3")

In [28]:
s3_object = s3.get_object(Bucket=BUCKET_NAME, Key=raw_data_filename)

data = s3_object["Body"].read().decode("utf-8")
df = pd.read_csv(StringIO(data))

s3_object = s3.get_object(Bucket=BUCKET_NAME, Key=raw_inference_data_filename)

data = s3_object["Body"].read().decode("utf-8")
inference_df = pd.read_csv(StringIO(data))

df.head(), inference_df.head()

,Datetime,Open,High,Low,Close,Adj Close,Volume
0,2024-05-15 09:30:00-04:00,5263.259766,5271.049805,5263.259766,5270.939941,5270.939941,0
1,2024-05-15 09:31:00-04:00,5271.250000,5273.240234,5271.149902,5271.439941,5271.439941,12862437
2,2024-05-15 09:32:00-04:00,5271.779785,5275.759766,5271.779785,5274.950195,5274.950195,11405230
3,2024-05-15 09:33:00-04:00,5275.160156,5277.259766,5273.870117,5276.120117,5276.120117,8963509
4,2024-05-15 09:34:00-04:00,5275.979980,5276.200195,5274.560059,5276.109863,5276.109863,7163548


In [4]:
df = pd.read_csv("../" + raw_data_filename)
inference_df = pd.read_csv("../" + raw_inference_data_filename)
df.head(), inference_df.head()

(                    Datetime         Open         High          Low  \
 0  2024-05-17 09:30:00-04:00  5303.100098  5305.450195  5302.660156   
 1  2024-05-17 09:31:00-04:00  5303.060059  5303.520020  5301.149902   
 2  2024-05-17 09:32:00-04:00  5301.410156  5301.720215  5299.410156   
 3  2024-05-17 09:33:00-04:00  5300.279785  5301.419922  5299.569824   
 4  2024-05-17 09:34:00-04:00  5299.700195  5299.779785  5298.250000   
 
          Close    Adj Close   Volume  
 0  5303.569824  5303.569824        0  
 1  5301.330078  5301.330078  6465998  
 2  5300.509766  5300.509766  6910800  
 3  5299.649902  5299.649902  6913787  
 4  5299.040039  5299.040039  5945542  ,
                     Datetime         Open         High          Low  \
 0  2024-06-14 15:29:00-04:00  5427.899902  5428.529785  5427.779785   
 1  2024-06-14 15:30:00-04:00  5428.410156  5428.470215  5426.439941   
 2  2024-06-14 15:31:00-04:00  5426.609863  5427.770020  5426.339844   
 3  2024-06-14 15:32:00-04:00  5427.3

In [5]:
df = df[["Datetime", "Close"]]
inference_df = inference_df[["Datetime", "Close"]]
df.head()

,Datetime,Close
0,2024-05-17 09:30:00-04:00,5303.569824
1,2024-05-17 09:31:00-04:00,5301.330078
2,2024-05-17 09:32:00-04:00,5300.509766
3,2024-05-17 09:33:00-04:00,5299.649902
4,2024-05-17 09:34:00-04:00,5299.040039


In [6]:
# Ensure data is sorted by datetime
df = df.sort_values(by=["Datetime"])
inference_df = inference_df.sort_values(by=["Datetime"])

In [7]:
df["DayOfWeek"] = pd.to_datetime(df["Datetime"]).dt.dayofweek
df["Hour"] = pd.to_datetime(df["Datetime"]).dt.hour
df["Minute"] = pd.to_datetime(df["Datetime"]).dt.minute

inference_df["DayOfWeek"] = pd.to_datetime(inference_df["Datetime"]).dt.dayofweek
inference_df["Hour"] = pd.to_datetime(inference_df["Datetime"]).dt.hour
inference_df["Minute"] = pd.to_datetime(inference_df["Datetime"]).dt.minute

In [8]:
def one_hot_encode_day_of_week(df, max_day_of_week=4):
    for i in range(max_day_of_week + 1):
        df[f"DayOfWeek_{i}"] = (df["DayOfWeek"] == i).astype(int)
    df.drop(columns=["DayOfWeek"], inplace=True)
    return df


one_hot_encode_day_of_week(inference_df)
one_hot_encode_day_of_week(df)

,Datetime,Close,Hour,Minute,DayOfWeek_0,DayOfWeek_1,DayOfWeek_2,DayOfWeek_3,DayOfWeek_4
0,2024-05-17 09:30:00-04:00,5303.569824,9,30,0,0,0,0,1
1,2024-05-17 09:31:00-04:00,5301.330078,9,31,0,0,0,0,1
2,2024-05-17 09:32:00-04:00,5300.509766,9,32,0,0,0,0,1
3,2024-05-17 09:33:00-04:00,5299.649902,9,33,0,0,0,0,1
4,2024-05-17 09:34:00-04:00,5299.040039,9,34,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...
7711,2024-06-14 15:55:00-04:00,5425.430176,15,55,0,0,0,0,1
7712,2024-06-14 15:56:00-04:00,5427.629883,15,56,0,0,0,0,1
7713,2024-06-14 15:57:00-04:00,5427.450195,15,57,0,0,0,0,1
7714,2024-06-14 15:58:00-04:00,5429.200195,15,58,0,0,0,0,1


In [9]:
df = df.drop(columns=["Datetime"])
inference_df = inference_df.drop(columns=["Datetime"])

In [10]:
total_size = len(df)
# test_size = int(0.1 * total_size)
# validation_size = int(0.1 * total_size)
test_size = 390
validation_size = 390
train_size = total_size - test_size - validation_size

test_start_idx = total_size - test_size
validation_start_idx = test_start_idx - validation_size

train_df = df.iloc[:validation_start_idx]
validation_df = df.iloc[validation_start_idx:test_start_idx]
test_df = df.iloc[test_start_idx:]

In [11]:
print("Training set size:", len(train_df))
print("Validation set size:", len(validation_df))
print("Test set size:", len(test_df))
print("Inference data size:", len(inference_df))

# Verify the splits
print("\nTrain DataFrame:")
print(train_df.head())
print("\nValidation DataFrame:")
print(validation_df.head())
print("\nTest DataFrame:")
print(test_df.head())
print("\nInference DataFrame:")
print(inference_df.head())

Training set size: 6936
Validation set size: 390
Test set size: 390
Inference data size: 31

Train DataFrame:
         Close  Hour  Minute  DayOfWeek_0  DayOfWeek_1  DayOfWeek_2  \
0  5303.569824     9      30            0            0            0   
1  5301.330078     9      31            0            0            0   
2  5300.509766     9      32            0            0            0   
3  5299.649902     9      33            0            0            0   
4  5299.040039     9      34            0            0            0   

   DayOfWeek_3  DayOfWeek_4  
0            0            1  
1            0            1  
2            0            1  
3            0            1  
4            0            1  

Validation DataFrame:
            Close  Hour  Minute  DayOfWeek_0  DayOfWeek_1  DayOfWeek_2  \
6936  5436.379883     9      30            0            0            0   
6937  5435.600098     9      31            0            0            0   
6938  5435.049805     9      32       

In [12]:
scaler = MinMaxScaler()

train_df.loc[:, "Close"] = scaler.fit_transform(train_df[["Close"]])

validation_df.loc[:, "Close"] = scaler.transform(validation_df[["Close"]])

test_df.loc[:, "Close"] = scaler.transform(test_df[["Close"]])

inference_df.loc[:, "Close"] = scaler.transform(inference_df[["Close"]])

train_df.head()

,Close,Hour,Minute,DayOfWeek_0,DayOfWeek_1,DayOfWeek_2,DayOfWeek_3,DayOfWeek_4
0,0.436134,9,30,0,0,0,0,1
1,0.427299,9,31,0,0,0,0,1
2,0.424063,9,32,0,0,0,0,1
3,0.420671,9,33,0,0,0,0,1
4,0.418265,9,34,0,0,0,0,1


In [13]:
# store later as metadata in feature store
# if scaler is MinMaxScaler
if type(scaler) == MinMaxScaler:
    scaler_params = {
        "data_min": scaler.data_min_[0],
        "data_max": scaler.data_max_[0],
        "data_range": scaler.data_range_[0],
        "min_": scaler.min_[0],
        "scale_": scaler.scale_[0],
    }
elif type(scaler) == StandardScaler:
    scaler_params = {
        "mean": scaler.mean_[0],
        "variance": scaler.var_[0],
        "scale_": scaler.scale_[0],
    }
else:
    raise ValueError("Unsupported scaler type")
scaler_params

{'data_min': 5193.009765625,
 'data_max': 5446.509765625,
 'data_range': 253.5,
 'min_': -20.48524562376726,
 'scale_': 0.0039447731755424065}

In [15]:
%mkdir -p ../data/processed

In [17]:
train_df.to_csv("../data/processed/train.csv", index=False)
validation_df.to_csv("../data/processed/validation.csv", index=False)
test_df.to_csv("../data/processed/test.csv", index=False)
inference_df.to_csv("../data/processed/inference.csv", index=False)

In [107]:
# REVISIT: Maybe prepare the dataset in a format that needs to be passed to the model

In [53]:
# REVISIT: use feature store later instead of plain s3, version things and store
# scaler params as metadata in feature store


def upload_df_to_s3(df, bucket_name, object_name):
    csv_buffer = StringIO()
    df.to_csv(csv_buffer, index=False)
    s3.put_object(Bucket=bucket_name, Key=object_name, Body=csv_buffer.getvalue())


root_folder = "data/processed"

upload_df_to_s3(train_df, BUCKET_NAME, f"{root_folder}/train.csv")
upload_df_to_s3(validation_df, BUCKET_NAME, f"{root_folder}/validation.csv")
upload_df_to_s3(test_df, BUCKET_NAME, f"{root_folder}/test.csv")
upload_df_to_s3(inference_df, BUCKET_NAME, f"{root_folder}/inference.csv")

In [18]:
scaler_params_json = json.dumps(scaler_params)

In [19]:
with open("../data/processed/scaler_params.json", "w") as f:
    f.write(scaler_params_json)

In [59]:
s3.put_object(
    Bucket=BUCKET_NAME, Key=f"{root_folder}/scaler_params.json", Body=scaler_params_json
)

{'ResponseMetadata': {'RequestId': 'ZQF23Z0WFZ3B9GKG',
  'HostId': 'mCDKcUIqSm7YocRgCn+at3h2ojChzibqcWZbssSZPr1c6QYBX2JuH0GE8NyKjMSKTzss5aC+ToS8x7YDVe1DEQ==',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'mCDKcUIqSm7YocRgCn+at3h2ojChzibqcWZbssSZPr1c6QYBX2JuH0GE8NyKjMSKTzss5aC+ToS8x7YDVe1DEQ==',
   'x-amz-request-id': 'ZQF23Z0WFZ3B9GKG',
   'date': 'Fri, 14 Jun 2024 10:00:17 GMT',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"3ca7802d62b31cccc7b07fa80d4d8d3d"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"3ca7802d62b31cccc7b07fa80d4d8d3d"',
 'ServerSideEncryption': 'AES256'}

Only to test that upload worked

In [55]:
s3.list_objects(Bucket=BUCKET_NAME)["Contents"]

[{'Key': 'data/processed/test.csv',
  'LastModified': datetime.datetime(2024, 6, 14, 9, 57, 52, tzinfo=tzlocal()),
  'ETag': '"4d1370a95a9260cfba866dcc4f51791c"',
  'Size': 36263,
  'StorageClass': 'STANDARD',
  'Owner': {'ID': 'ec1ed018b5c0ee51b7177c58aeefc36b88bc97ed583ad096499121407c50d423'}},
 {'Key': 'data/processed/train.csv',
  'LastModified': datetime.datetime(2024, 6, 14, 9, 57, 52, tzinfo=tzlocal()),
  'ETag': '"1e087f7f9658fc8c539bcfb3ce575d20"',
  'Size': 292145,
  'StorageClass': 'STANDARD',
  'Owner': {'ID': 'ec1ed018b5c0ee51b7177c58aeefc36b88bc97ed583ad096499121407c50d423'}},
 {'Key': 'data/processed/validation.csv',
  'LastModified': datetime.datetime(2024, 6, 14, 9, 57, 52, tzinfo=tzlocal()),
  'ETag': '"9767fe903678ae5cb4bf72847b456203"',
  'Size': 36364,
  'StorageClass': 'STANDARD',
  'Owner': {'ID': 'ec1ed018b5c0ee51b7177c58aeefc36b88bc97ed583ad096499121407c50d423'}},
 {'Key': 'data/raw/sp500.csv',
  'LastModified': datetime.datetime(2024, 6, 14, 7, 19, 40, tzinfo=

In [60]:
s3_object = s3.get_object(Bucket=BUCKET_NAME, Key=f"{root_folder}/train.csv")

loaded_data = s3_object["Body"].read().decode("utf-8")
loaded_df = pd.read_csv(StringIO(loaded_data))
loaded_df.head()

,Datetime,Close
0,2024-05-15 09:30:00-04:00,0.429912
1,2024-05-15 09:31:00-04:00,0.432670
2,2024-05-15 09:32:00-04:00,0.452035
3,2024-05-15 09:33:00-04:00,0.458489
4,2024-05-15 09:34:00-04:00,0.458433
